# Language Modelling on IPUs - Training

In this notebook, we'll see how to train a [🤗 Transformers](https://github.com/huggingface/transformers) model on a language modelling task. We will cover two types of language modelling tasks:

- Causal language modelling: The model has to predict the next token in the sentence (so the labels are the same as the inputs shifted to the right). To make sure the model does not cheat, it gets an attention mask that will prevent it from accessing the tokens after token `i` when trying to predict token `i+1` in the sentence.

![Widget inference representing the causal language modelling task](images/causal_language_modeling.png)

- Masked language modelling: The model has to predict some tokens that are masked in the input. It still has access to the whole sentence, so it can use the tokens before and after the tokens that have been masked to predict their value.

![Widget inference representing the masked language modelling task](images/masked_language_modeling.png)

We will see how to easily load and preprocess the dataset for each of these tasks, and how to use the `IPUTrainer` API to train a model on it.

This notebooks assumes you have trained a tokenizer on the corpus you are using (see the [How to train a tokenizer](https://github.com/huggingface/notebooks/blob/master/examples/tokenizer_training.ipynb) notebook for details).

|  Domain | Tasks | Model | Datasets | Workflow |   Number of IPUs   | Execution time |
|---------|-------|-------|----------|----------|--------------|--------------|
| Natural language processing | Causal language modelling and Masked language modelling | gpt2 and bert-base-cased | Wikitext 2 | Training | 4 or 16 | 28 min on POD4, 15 min on POD16 |

[![Join our Slack Community](https://img.shields.io/badge/Slack-Join%20Graphcore's%20Community-blue?style=flat-square&logo=slack)](https://www.graphcore.ai/join-community)


## Environment setup

The best way to run this demo is on Paperspace Gradient's cloud IPUs because everything is already set up for you.

[![Run on Gradient](https://assets.paperspace.io/img/gradient-badge.svg)](https://ipu.dev/414XiNp)

To run the demo using other IPU hardware, you need to have the Poplar SDK enabled. Refer to the [Getting Started guide](https://docs.graphcore.ai/en/latest/getting-started.html#getting-started) for your system for details on how to enable the Poplar SDK. Also refer to the [Jupyter Quick Start guide](https://docs.graphcore.ai/projects/jupyter-notebook-quick-start/en/latest/index.html) for how to set up Jupyter to be able to run this notebook on a remote IPU machine.

## Dependencies and configuration

In order to improve usability and support for future users, Graphcore would like to collect information about the
applications and code being run in this notebook. The following information will be anonymised before being sent to Graphcore:

- User progression through the notebook
- Notebook details: number of cells, code being run and the output of the cells
- Environment details

You can disable logging at any time by running `%unload_ext graphcore_cloud_tools.notebook_logging.gc_logger` from any cell.

Install the dependencies for this notebook.

In [ ]:
%pip install "optimum-graphcore==0.7"
%pip install graphcore-cloud-tools[logger]@git+https://github.com/graphcore/graphcore-cloud-tools
%load_ext graphcore_cloud_tools.notebook_logging.gc_logger

The cache directories can be configured through environment variables or directly in the notebook:

In [ ]:
import os

executable_cache_dir = os.getenv("POPLAR_EXECUTABLE_CACHE_DIR", "/tmp/exe_cache/") + "/language_modelling_from_scratch"

### Sharing your model with the community

You can share your model with the 🤗 community. You do this by completing the following steps:

1. Store your authentication token from the 🤗 website. [Sign up to 🤗](https://huggingface.co/join) if you haven't already.
2. Execute the following cell and input your username and password:

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

Then you need to install Git-LFS to manage large files:

In [ ]:
!apt install git-lfs

## Preparing the dataset

For each of the tasks, we will use the Wikitext 2 dataset as an example. You can load it easily with the 🤗 Datasets library.

In [ ]:
from datasets import load_dataset
datasets = load_dataset('wikitext', 'wikitext-2-raw-v1')

You can replace the dataset above with any dataset hosted on [🤗 Datasets](https://huggingface.co/datasets). 

You can also use your own data. Just uncomment the following cell and replace the paths shown with the paths to your files:

In [ ]:
# datasets = load_dataset("text", data_files={"train": path_to_train.txt, "validation": path_to_validation.txt}

You can also load datasets from a CSV or a JSON file. See the Datasets documentation for [loading datasets from local files](https://huggingface.co/docs/datasets/loading_datasets.html#from-local-files) for more information.

To access an actual element, you need to select a split ("train" in the example) and specify an index:

In [ ]:
datasets["train"][10]

We want to get a sense of what the data looks like, so we define the `show_random_elements` function to display some examples picked randomly from the dataset.

In [ ]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [ ]:
show_random_elements(datasets["train"])

As we can see, some of the text samples are a full paragraph of a Wikipedia article while others are just titles or empty lines.

## Causal language modelling

For causal language modelling (CLM) we are going to take all the text in our dataset and concatenate them after they are tokenized. Then we will split them in samples of a certain sequence length. This means that the model will receive chunks of contiguous text that may look like:
```
part of text 1
```
or 
```
end of text 1 [BOS_TOKEN] beginning of text 2
```
depending on whether the samples span over several of the original text samples in the dataset or not. The labels will be the same as the inputs, shifted to the left.

We will use the [`gpt2`](https://huggingface.co/gpt2) architecture for this example. You can pick any of the [🤗 models for causal language modelling](https://huggingface.co/models?filter=causal-lm) as long as that model is supported by Optimum Graphcore. The IPU config files of the supported models are available in Graphcore's [🤗 account](https://huggingface.co/Graphcore). You can also create your own IPU config file locally. For the tokenizer, you can replace the checkpoint with the one you trained yourself.

In this notebook, we are using both data parallelism and pipeline parallelism (see the [tutorial on efficient data loading](https://github.com/graphcore/examples/tree/master/tutorials/tutorials/pytorch/efficient_data_loading) for more information). Therefore the global batch size, which is the actual number of samples used for the weight update, is determined from three factors:
- global batch size = micro batch size * gradient accumulation steps * replication factor

The replication factor is determined by the type of IPU Pod used, which will be used as a key to select the replication factor from a dictionary defined in the IPU config file. For example, the dictionary in the IPU config file [Graphcore/gpt2-small-ipu](https://huggingface.co/Graphcore/gpt2-small-ipu/blob/main/ipu_config.json) looks like this:
- "replication_factor": {"pod4": 1, "pod8": 2, "pod16": 4, "pod32": 8, "pod64": 16, "default": 1}

Depending on your model and the IPU Pod you are using, you might need to adjust these three batch-size-related arguments.

In [ ]:
model_checkpoint = "gpt2"
tokenizer_checkpoint = "sgugger/gpt2-like-tokenizer"

ipu_config_name = "Graphcore/gpt2-small-ipu"
micro_batch_size = 1
gradient_accumulation_steps = 64
dataloader_workers = 64

To tokenize all our text samples with the same vocabulary that was used when training the model, we have to download a pre-trained tokenizer. This is all done by the `AutoTokenizer` class:

In [ ]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(tokenizer_checkpoint)

We can now call the tokenizer on all our text samples. This is very simple, using the [`map`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasets.Dataset.map) method from the Datasets library. First we define a function that calls the tokenizer on our texts:

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["text"])

Then we apply it to all the splits in our `datasets` object, using `batched=True` and 4 processes to speed up the preprocessing. We won't need the `text` column afterward, so we discard it.

In [ ]:
tokenized_datasets = datasets.map(tokenize_function, batched=True, num_proc=4, remove_columns=["text"])

If we now look at an element of our datasets, we will see the text has been replaced with `input_ids` that the model will need:

In [ ]:
tokenized_datasets["train"][1]

Now we need to concatenate all our text samples together then split the result into small chunks of a certain block size (`block_size`). To do this, we will use the `map` method again, with the option `batched=True`. This option lets us change the number of samples in the datasets by returning a different number of samples than we originally had. This means that we can create a new set of samples from an existing set of samples.

We can read the maximum length our model was pre-trained with (with tokenizer.model_max_length), but since the value might be too big to fit on your IPU RAM, we set it to 128.

In [ ]:
# block_size = tokenizer.model_max_length
block_size = 128

Then we write the preprocessing function that will group our text samples:

In [ ]:
def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
        # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

Note that we duplicate the inputs for our labels. This is because the model of the 🤗 Transformers library applies a shift to the right, so we don't need to do it manually.

Also note that by default, the `map` method will send a batch of 1,000 examples to be treated by the preprocessing function. So here, we will drop the remainder to make the concatenated tokenized text samples a multiple of `block_size` every 1,000 examples. You can adjust this behaviour by passing a larger batch size (which will also take longer to be processed). You can also speed up the preprocessing by using multiprocessing:

In [ ]:
lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4,
)

We can check our datasets have changed. Now the samples contain chunks of `block_size` contiguous tokens, potentially spanning over several of our original text samples.

In [ ]:
tokenizer.decode(lm_datasets["train"][1]["input_ids"])

To instantiate `IPUTrainer`, we will need to define:
* `IPUConfig`, which is a class that specifies attributes and configuration parameters to compile and put the model on the device.
* A model.
* `IPUTrainingArguments`, which is a class that contains all the attributes to customize the training.

We initialize `IPUConfig` with one config name or a path, which we set earlier. We also get the model configuration from the model name set earlier and initialize our model using that config.

In [ ]:
from transformers import AutoConfig, AutoModelForCausalLM
from optimum.graphcore import IPUConfig, IPUTrainer, IPUTrainingArguments

ipu_config = IPUConfig.from_pretrained(ipu_config_name, executable_cache_dir=executable_cache_dir)

config = AutoConfig.from_pretrained(model_checkpoint)
config.update({'activation_function':'gelu'})
model = AutoModelForCausalLM.from_config(config)

`IPUTrainingArguments` requires one folder name, which will be used to save the checkpoints of the model. All other arguments are optional:

In [ ]:
training_args = IPUTrainingArguments(
    f"{model_checkpoint}-wikitext2",
    learning_rate=2e-5,
    weight_decay=0.01,
    per_device_train_batch_size=micro_batch_size,
    per_device_eval_batch_size=micro_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    num_train_epochs=10,
    loss_scaling=16384,
    n_ipu=4,
    warmup_ratio=0.1,
    dataloader_drop_last=True,
    dataloader_num_workers=dataloader_workers,
    logging_steps=10,
    push_to_hub=False,
    # hub_model_id=f"username-or-organization/{model_checkpoint}-wikitext2",
)

`push_to_hub` and `hub_model_id` in `IPUTrainingArguments` are necessary if we want to push the model to the [🤗 Models Hub](https://huggingface.co/models) regularly during training. You can remove them if you didn't follow the installation steps at the beginning of this notebook. If you want to save your model locally to a name that is different to the name of the repository it will be pushed to, or if you want to push your model under an organization and not your name space, use the `hub_model_id` argument to set the repo name (it needs to be the full name, including your namespace: for instance `"sgugger/gpt-finetuned-wikitext2"` or `"huggingface/gpt-finetuned-wikitext2"`).

Finally, we pass along all of these to the `IPUTrainer` class:

In [ ]:
from transformers import default_data_collator

trainer = IPUTrainer(
    model=model,
    ipu_config=ipu_config,
    args=training_args,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["validation"],
    tokenizer=tokenizer,
    data_collator=default_data_collator,
)

We are now ready to train our model:

In [ ]:
trainer.train()

Once the training is complete, we can evaluate our model and get its perplexity on the validation set like this:

In [ ]:
import math
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

The perplexity is still quite high since we only trained on a small dataset for a small number of epochs. For a real language model training, you would need a larger dataset and more epochs.

You can now upload the result of the training to the 🤗 Hub:

In [ ]:
# trainer.push_to_hub()

You can now share this model and other users can load it with the identifier `"your-username/the-name-you-picked"` so for instance:

```python
from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained("sgugger/my-awesome-model")
```

## Masked language modelling

For masked language modelling (MLM) we are going to use the same preprocessing as before for our dataset with one additional step: we will randomly mask some tokens (by replacing them with `[MASK]` and the labels will be adjusted to only include the masked tokens (we don't have to predict the non-masked tokens). If you use a tokenizer you trained yourself, make sure the `[MASK]` token is among the special tokens you passed during training!

We will use the [`bert-base-cased`](https://huggingface.co/bert-based-cased) model for this example.  You can pick any of the [🤗 models for masked language modelling](https://huggingface.co/models?filter=masked-lm) as long as that model is supported by Optimum Graphcore. The IPU config files of the supported models are available in Graphcore's [🤗 account](https://huggingface.co/Graphcore). You can also create your own IPU config file locally. For the tokenizer, replace the checkpoint with the one you trained.

In [ ]:
model_checkpoint = "bert-base-cased"
tokenizer_checkpoint = "sgugger/bert-like-tokenizer"

ipu_config_name = "Graphcore/bert-base-ipu"

We can apply the same tokenization function as before, we just need to update our tokenizer to use the model checkpoint we just picked:

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(tokenizer_checkpoint)
tokenized_datasets = datasets.map(tokenize_function, batched=True, num_proc=4, remove_columns=["text"])

As with the causal language modelling example, we group the text samples together and create chunks of length `block_size`. You can skip that step if your dataset is composed of individual sentences.

In [ ]:
lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4,
)

The rest is very similar to what we did for causal language modelling, with two exceptions:
* We need a model suitable for masked language modelling.
* We need a special data collator.

First, we use a model suitable for masked language modelling:

In [ ]:
from transformers import AutoConfig, AutoModelForMaskedLM
from optimum.graphcore import IPUConfig, IPUTrainer, IPUTrainingArguments

ipu_config = IPUConfig.from_pretrained(ipu_config_name, executable_cache_dir=executable_cache_dir)

config = AutoConfig.from_pretrained(model_checkpoint)
model = AutoModelForMaskedLM.from_config(config)

We redefine the `IPUTrainingArguments` class:

In [ ]:
training_args = IPUTrainingArguments(
    f"{model_checkpoint}-wikitext2-test-mlm",
    learning_rate=2e-5,
    weight_decay=0.01,
    per_device_train_batch_size=micro_batch_size,
    per_device_eval_batch_size=micro_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    num_train_epochs=10,
    dataloader_drop_last=True,
    dataloader_num_workers=dataloader_workers,
    warmup_ratio=0.1,
    logging_steps=10,
    n_ipu=4,
    push_to_hub=False,
    # hub_model_id=f"username-or-organization/{model_checkpoint}-wikitext2-test-mlm",
)

Like before, the last two arguments in `IPUTrainingArguments` are needed if we want to push the model to the [🤗 Models Hub](https://huggingface.co/models) at the end of training. Remove these two arguments if you didn't follow the installation steps at the top of the notebook. If you want to save your model locally to a name that is different to the name of the repository it will be pushed to, or if you want to push your model under an organization and not your name space, use the `hub_model_id` argument to set the repo name (it needs to be the full name, including your namespace: for instance `"sgugger/gpt-finetuned-wikitext2"` or `"huggingface/gpt-finetuned-wikitext2"`).

Finally, we use a special data collator. The data collator is a function that is responsible for taking samples and batching them into tensors. In the causal language modelling example, we didn't need anything special, so we just used the default data collator. Here we want to randomly mask the data. We could do it as a pre-processing step (like with the tokenization) but then the tokens would always be masked the same way at each epoch. By doing this step inside the data collator, we ensure this random masking is done in a new way each time we go over the data.

To do this masking, we use `DataCollatorForLanguagemodelling` which lets us adjust the probability of the masking:

In [ ]:
from transformers import DataCollatorForLanguageModeling
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)

Then we just have to pass everything to `IPUTrainer` and begin training:

In [ ]:
trainer = IPUTrainer(
    model=model,
    args=training_args,
    ipu_config=ipu_config,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["validation"],
    data_collator=data_collator,
)

In [ ]:
trainer.train()

Like before, we can evaluate our model on the validation set. The perplexity is much lower than for the CLM objective because for the MLM objective, we only have to make predictions for the masked tokens (which represent 15% of the total here) while having access to the rest of the tokens. It's thus an easier task for the model.

In [ ]:
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

The perplexity is still quite high since we only trained on a small dataset for a small number of epochs. For a real language model training, you  would need a larger dataset and more epochs.

You can now upload the result of the training to the 🤗 Hub:

In [ ]:
# trainer.push_to_hub()

You can also share this model and other users can load it with the identifier `"your-username/the-name-you-picked"` so for instance:

```python
from transformers import AutoModelForMaskedLM

model = AutoModelForMaskedLM.from_pretrained("sgugger/my-awesome-model")
```

## Next steps

Check out the full list of [IPU-powered Jupyter Notebooks](https://www.graphcore.ai/ipu-jupyter-notebooks) to get more of a feel for how IPUs perform on other tasks.